In [17]:
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import numpy as np

# K.set_image_dim_ordering('th')
K.set_image_dim_ordering('tf')

path_to_data = '/home/cody/code/pokemon_classifier/dataset'
path_to_validation_data = '/home/cody/code/pokemon_classifier/validation_dataset'
model_path = '/home/cody/code/pokemon_classifier/models'
batch_size = 30

### Train Model

In [2]:
base_model = Xception(input_shape=(299, 299, 3), include_top=False, weights='imagenet')

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    path_to_data,  # this is the target directory
    target_size=(299, 299),  # all images will be resized to 299x299
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)


Found 8971 images belonging to 151 classes.


In [4]:
# Get the indices, we'll use these when testing (associate output with pokemon name)
class_indices = train_generator.class_indices

import json
with open(model_path + '/class_indices.json', 'w') as outfile:
    json.dump(class_indices, outfile)

In [5]:
validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_generator = validation_datagen.flow_from_directory(
    path_to_validation_data,
    target_size=(299, 299),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)

Found 1397 images belonging to 151 classes.


In [7]:
# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(151, activation='softmax')(x)

# add your top layer block to your base model
model = Model(base_model.input, predictions)

print(model.summary())

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', patience=5, verbose=0)
]

# fine-tune the model first only top layer
model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        use_multiprocessing=True,
        epochs=16,
        # epochs=100,
        validation_data=validation_generator,
        validation_steps = validation_generator.samples // batch_size,
        callbacks=callbacks_list
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2403s 8s/step - loss: 3.2704 - acc: 0.3351 - val_loss: 3.0650 - val_acc: 0.3319

Epoch 00001: acc improved from -inf to 0.33620, saving model to /home/cody/code/pokemon_classifier/models/top_model_weights.h5
Epoch 2/16


/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2402s 8s/step - loss: 1.6905 - acc: 0.6374 - val_loss: 2.8636 - val_acc: 0.3529

Epoch 00002: acc improved from 0.33620 to 0.63941, saving model to /home/cody/code/pokemon_classifier/models/top_model_weights.h5
Epoch 3/16
 38/299 [==>...........................] - ETA: 30:20 - loss: 1.3350 - acc: 0.7053

...

299/299 [==============================] - 2408s 8s/step - loss: 0.3921 - acc: 0.9049 - val_loss: 3.0560 - val_acc: 0.3804

Epoch 00014: acc improved from 0.90359 to 0.90784, saving model to /home/cody/code/pokemon_classifier/models/top_model_weights.h5
Epoch 15/16
114/299 [==========>...................] - ETA: 21:27 - loss: 0.3214 - acc: 0.9231

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2407s 8s/step - loss: 0.3516 - acc: 0.9096 - val_loss: 3.0169 - val_acc: 0.3978

Epoch 00015: acc improved from 0.90784 to 0.91254, saving model to /home/cody/code/pokemon_classifier/models/top_model_weights.h5
Epoch 16/16
147/299 [=============>................] - ETA: 17:30 - loss: 0.3515 - acc: 0.9138

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2407s 8s/step - loss: 0.3501 - acc: 0.9127 - val_loss: 3.1281 - val_acc: 0.3993

Epoch 00016: acc improved from 0.91254 to 0.91567, saving model to /home/cody/code/pokemon_classifier/models/top_model_weights.h5


In [8]:
model.load_weights(top_weights_path)

In [9]:
based_model_last_block_layer_number = 126

for layer in model.layers[:based_model_last_block_layer_number]:
    layer.trainable = False
for layer in model.layers[based_model_last_block_layer_number:]:
    layer.trainable = True

model.compile(optimizer='nadam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='loss', patience=5, verbose=0)
]

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    use_multiprocessing=True,
    shuffle=True,
    epochs=16,
    validation_data=validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    callbacks=callbacks_list
)

Epoch 1/16
 31/299 [==>...........................] - ETA: 33:47 - loss: 1.1342 - acc: 0.7022

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2557s 9s/step - loss: 1.3948 - acc: 0.6518 - val_loss: 4.2311 - val_acc: 0.4080

Epoch 00001: acc improved from -inf to 0.65395, saving model to /home/cody/code/pokemon_classifier/models/model_weights.h5
Epoch 2/16
156/299 [==============>...............] - ETA: 17:46 - loss: 0.6391 - acc: 0.8259

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2556s 9s/step - loss: 0.7446 - acc: 0.8031 - val_loss: 3.8726 - val_acc: 0.4341

Epoch 00002: acc improved from 0.65395 to 0.80573, saving model to /home/cody/code/pokemon_classifier/models/model_weights.h5
Epoch 3/16
111/299 [==========>...................] - ETA: 23:25 - loss: 0.4386 - acc: 0.8688

...

299/299 [==============================] - 2561s 9s/step - loss: 0.1922 - acc: 0.9440 - val_loss: 3.1766 - val_acc: 0.5181

Epoch 00014: acc did not improve from 0.94922
Epoch 15/16
 14/299 [>.............................] - ETA: 35:21 - loss: 0.1545 - acc: 0.9500

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2560s 9s/step - loss: 0.1776 - acc: 0.9472 - val_loss: 2.8268 - val_acc: 0.5312

Epoch 00015: acc improved from 0.94922 to 0.95023, saving model to /home/cody/code/pokemon_classifier/models/model_weights.h5
Epoch 16/16
 74/299 [======>.......................] - ETA: 28:03 - loss: 0.1482 - acc: 0.9554

/usr/local/lib/python2.7/dist-packages/PIL/Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


299/299 [==============================] - 2561s 9s/step - loss: 0.1631 - acc: 0.9501 - val_loss: 2.9867 - val_acc: 0.5101

Epoch 00016: acc improved from 0.95023 to 0.95314, saving model to /home/cody/code/pokemon_classifier/models/model_weights.h5


### Save Model

In [15]:
# save model as architecture and weights
model_json = model.to_json()
with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
    json_file.write(model_json)

In [11]:
# save whole thing
model.save('models/my_model.h5')

### Load Model

In [ ]:
# load model as architecture and weights

from keras.models import model_from_json

json_file = open(model_path + "/model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()

test_model = model_from_json(loaded_model_json)

test_model.load_weights(model_path + '/model_weights.h5')

In [19]:
# load whole thing
test_model = load_model('models/my_model.h5')

### Test Model

In [40]:

img = load_img(path_to_data + '/Starmie/b05d5dc1b3364dc8ac42f593ad3a479f.jpg', target_size=(299,299))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)

pred = model.predict(x)
np.argmax(pred, axis=1)

array([72])